In [1]:
import xcdat
import os
import xarray as xr
import dash
import geopandas as gpd
import regionmask
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash
import plotly
import plotly.graph_objects as go
import plotly.express as px
states_file = '/home/goodnight1/shapefiles/cb_2018_us_state_20m.shp' # https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
global states_df
states_df = gpd.read_file(states_file)
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import time
import json
xr.set_options(keep_attrs=True)
import pandas as pd

## Creating a Choropleth map

In [2]:
# NCA4 region dictionary with STUPS codes

nca4_regions = {'Northern Great Plains': ['MT', 'NE', 'SD', 'ND', 'WY'], \
            'Southern Great Plains': ['TX', 'OK', 'KS'], \
            'Southwest': ['CA', 'NV', 'AZ', 'NM', 'CO', 'UT'], \
            'Southeast': ['FL', 'GA', 'SC', 'MS', 'AL', 'AR', 'TN', 'KY', 'NC', 'VA', 'LA'], \
            'Northeast': ['WV', 'DC', 'PA', 'MA', 'NY', 'CT', 'RI', 'ME', 'NJ', 'VT', 'NH', 'MD', 'DE'], \
            'Northwest': ['WA', 'OR', 'ID'], \
            'Midwest':   ['IL', 'MI', 'WI', 'MO', 'IA', 'IN', 'OH', 'MN']}
            #'Alaska':['AK'], \
            #'Hawai\'i and Pacific Islands':['HI'], #, 'AS', 'GU', 'MP'], 
            #'Carribean': ['PR', 'VI']}
            
half_regions = {'East': ['WV', 'DC', 'PA', 'MA', 'NY', 'CT', 'RI', 'ME', 'NJ', 'VT', 'NH', 'MD', 'DE', 'IL', 'MI', 'WI', 'MO', 'IA', 'IN', 'OH', 'MN', 'FL', 'GA', 'SC', 'MS', 'AL', 'AR', 'TN', 'KY', 'NC', 'VA', 'LA'], 
                'West': ['MT', 'NE', 'SD', 'ND', 'WY', 'TX', 'OK', 'KS', 'WA', 'OR', 'ID', 'CA', 'NV', 'AZ', 'NM', 'CO', 'UT']
               }



In [3]:
# adding an NCA4 region column to geoDF and creating a mask
nca_state = []
region_names = []
all_states = list(states_df['STUSPS'])

for state in states_df['STUSPS']:
    for region in nca4_regions.keys(): 
        if state in nca4_regions[region]: 
            all_states.remove(state)
            nca_state.append(region)
            
for not_contained_state in all_states:
    states_df = states_df[states_df['STUSPS'] != not_contained_state]
    
half_state = []
all_states = list(states_df['STUSPS'])

for state in states_df['STUSPS']:
    for region in half_regions.keys(): 
        if state in half_regions[region]: 
            all_states.remove(state)
            half_state.append(region)
# for not_contained_state in all_states:
#     states_df = states_df[states_df['STUSPS'] != not_contained_state]
            
## new columns containing the region (NCA4, Half, or Continental US) each state is a part of
states_df['NCA4_region'] = nca_state   
states_df['half_region'] = half_state 
states_df['continental'] = ['cus']*len(half_state)

# Regional State, and half mask
regional_df = states_df.dissolve(by = 'NCA4_region')
regional_df['NAME'] = regional_df.index
regional_df = regional_df.reset_index(drop = True)

nca_mask    = regionmask.from_geopandas(regional_df, names = "NAME", name = "NCA4 regions")  # creates a regionmask -> 7 regions
state_mask  = regionmask.from_geopandas(states_df, names = "STUSPS", name = "States")        # creates a regionmask -> 48 regions (/per state)

half_df = states_df.dissolve(by = 'half_region')
half_df['NAME'] = half_df.index
half_df = half_df.reset_index(drop = True)

cus_df = states_df.dissolve(by = "continental")
cus_df ['NAME'] = cus_df.index
cus_df = cus_df.reset_index(drop = True)

gpd_dict = {'NCA4 Region':regional_df, 'States':states_df, 'Half': half_df, 'cus': cus_df}

half_mask = regionmask.from_geopandas(half_df, names = "NAME", name = "Half Regions")       # region mask for East/West regions
cus_mask  = regionmask.from_geopandas(cus_df, names = "NAME", name = "Continental US")      # region mask for full CUS region 

# Continental US mask
# us = gpd.read_file('/home/goodnight1/shapefiles/cb_2018_us_nation_20m.shp')
# us_mask = regionmask.from_geopandas(us, names = "NAME", name = "United States")

region_dict = {'NCA4 Region': nca_mask, "States": state_mask, 'Half': half_mask, 'cus':cus_mask} # 'United States': us_mask,

In [4]:
# creating plotly choropleth maps for each region type, for use within the dashboard. 
# storing each map in a dicionary, so that the data can be passed as storage within the dashboard layout and accessed later



choro_dict = {}

for regionType in list(gpd_dict.keys())[::-1]:
    
    ## figuring out which column in the geopandas dataframe we need to use as the hover/label data
    if ('NCA' in regionType) or ('Half' in regionType): 
        ident = "NAME"
    elif 'States' in regionType:
        ident = "STUSPS"
    elif "cus" in regionType: 
        ident = "NAME"
        
    df = gpd_dict[regionType]
    
    # coloring the map a uniform color based on a new dataframe column 'selection'; for later interactive use within dashboard

    df['selection'] = ['Not Selected']*len(df)
    
    fig_map = px.choropleth(df,
                           geojson=gpd_dict[regionType].geometry,
                           locations=gpd_dict[regionType].index,
                           labels = [ident],
                           title = None, 
                           projection = 'mercator',
                           custom_data=[ident],
                           hover_name = ident,
                           hover_data = [],
                           color = df.selection,
                           color_discrete_map={'Selected':'royalblue', 'Not Selected':'powderblue'}, 
                        ).update_geos(lataxis_range = [23, 55], 
                                      lonaxis_range = [-130, -60]).update_layout(showlegend=False, geo=dict(bgcolor= 'rgba(0,0,0,0)'), margin = {"r":0,"t":0,"l":0,"b":0}).update_traces(hovertemplate="")#, clickmode = "select", hovermode = False)
    choro_dict[regionType] = fig_map

In [5]:
"""
Accessing a list of methods (LOCA2 vs STAR)
Accessing a list of comparison Datasets (PRISM vs Livneh/NClimGrid)
Accessing a list of variable names (to be cleaned in the following cell)
"""

diri = './assets/fullFiles/'
files = [i for i in os.listdir(diri)]                      # stats data that we have access to 
methodList   = np.unique([i.split('.')[0] for i in files])
compList     = np.unique([i.split('.')[1] for i in files])
variableList = np.unique([i.split('.')[2] for i in files])

In [6]:
"""
Variable names associated withuantile and seasonal statistics data are currently named "quantile_{variable_name}" or "seasonal_{variable_name}" in the file names
We therefore need to create a new variable name depending on which quantile (e.g. 50th percentile) or season (e.g. JJA)
"""


newVariableList = []
# lists of all possible season and quantile options\

season_list = ['DJF', 'MAM', 'JJA', 'SON']
q_list      = ['q01', 'q05', 'q10', 'q25', 'q50', 'q75', 'q90', 'q95', 'q99', 'q99p9']


for variable in variableList: 
    
    if 'seasonal' in variable: 
        for season in season_list:                     # replace "seasonal" with four options for different season; append each to the variable list
            newVariableList.append(variable.replace('seasonal_', '').replace('seasonal', '') + '_' + season)
            
    elif (variable == 'pr') or (variable == 'tasmax'): # append all quantile options to the variable name and append each
        for q in q_list: 
            newVariableList.append(q + '_' + variable)
            
    else:                                              # append the old variable name otherwise
        newVariableList.append(variable)
        


In [7]:
diri = './assets/fullFiles/'

In [8]:
"""
We have different model data depending on the method (LOCA vs STAR). Here we define the lists of models that we have data for depending on the method
"""

loca2_files  = [i for i in os.listdir(diri) if ('loca2' in i)]
star_files   = [i for i in os.listdir(diri) if ('star' in i)]

modelListLoca = np.unique([i.split('.')[3] for i in loca2_files])
modelListStar = np.unique([i.split('.')[3] for i in star_files])

In [9]:
regionTypes = ['nca4', 'states', 'half', 'cus'] # user options in dashboard

In [10]:
import dash
from dash import dcc, ctx
from dash import html
from dash import dash_table

from dash import Input, Output, State, clientside_callback
from dash import ctx

In [22]:
"""
We need placeholder graphs to put in the dashboard. Initially these will not be visible to the user by setting {"display":"none"} in the style options.
"""

dummy_table = dash_table.DataTable(id = 'tbl', sort_action='native', style_cell = {'font_size': '18px'}, tooltip_duration = None)
dummy_graph = go.Figure(data = go.Scatter(x = [], y = [], hovertext =[], mode = 'markers')).update_layout(width = 800, height = 600, plot_bgcolor = '#ededed').update_xaxes(zerolinecolor = 'black', gridcolor = 'white').update_yaxes(zerolinecolor = 'black',gridcolor='white')
dummy_graph  = dummy_graph.add_trace(go.Scatter(x = [], y = [], mode = 'lines', line = dict(width = 2, color = 'black')))

In [51]:
app = JupyterDash()

In [52]:
app.layout = html.Div([
    html.H1('US Climate Data Product Statistics Dashboard', style = {'textAlign':'center', 'fontSize':'50px'}), # title
    
    html.Div(className = 'navigation-bar', children = [ # nav bar holder
        dcc.Graph(figure = fig_map, config={'displayModeBar':False}, id = 'usmap', style = {'position': 'relative', 'width':'20%'}), # click-able map (region select) 
        html.Div(children = [
            dcc.RadioItems(regionTypes, regionTypes[0], id = 'dropType', style = {'position': 'relative', 'width':'5%', 'fontSize':'20px', 'marginTop':'2%'}), # region type options (NCA4, States, Half, CUS)

            html.Div(children = [ # metric select
                html.H3('Metric', style = {'display': 'flex', 'flexDirection':'row', 'justifyContent':'center'}), 
                html.Div(children = [dcc.Dropdown(['rms', 'rmsc', 'mae', 'bias', 'corr'], 'rms', multi = False, id = 'dropMetric', className = 'drop-metric', style = {'position': 'relative', 'fontSize': '15px'})], 
                         style = {'alignItems':'start'})
            ], style = {'position':'relative', 'width':'10%','marginLeft':'1%'}),
            
            html.Div(children = [ # table type select ( Variable: compare variable difference w/in the same comparison dataset, Method: compare one variable w/in different methods/comparison datasets
                html.H3('Table Type', style = {'display': 'flex', 'flexDirection':'row', 'justifyContent':'center'}), 
                html.Div(children = [dcc.Dropdown(['Variable', 'Method'], 'Variable', multi = False, id = 'dropView', className = 'drop-view', style = {'position': 'relative', 'fontSize': '15px'})], 
                         style = {'alignItems':'start'})
            ], style = {'position':'relative', 'width':'10%','marginLeft':'1%'}),
            
            html.Div(children = [ # method select ( only used when table type = 'Variable') 
                html.H3('Method', style = {'display': 'flex', 'flexDirection':'row', 'justifyContent':'center'}), 
                html.Div(children = [dcc.Dropdown(['loca2', 'star'], 'loca2', multi = False, id = 'dropMethod', className = 'drop-method', style = {'position': 'relative', 'fontSize': '15px'})], 
                         style = {'alignItems':'start'})
            ], id = 'methodHolder', style = {'position':'relative', 'width':'10%', 'marginLeft':'1%'}),
            html.Div(children = [ # comparison dataset select ( only used when table type = 'Variable') 
                html.H3('Comparison DS', style = {'display': 'flex', 'flexDirection':'row', 'justifyContent':'center'}), 
                html.Div(children = [dcc.Dropdown(['prism', 'livneh', 'NClimGrid'], 'prism', multi = False, id = 'dropComp', className = 'drop-comp', style = {'position':'relative', 'fontSize': '15px'})], 
                         style = {'alignItems':'start'})
            ], id = 'compHolder', style = {'position':'relative', 'width':'15%', 'marginLeft':'1%'}),
            html.Div(children = [# variable select ( only used when table type = 'Variable') 
                html.H3('Variables', style = {'display': 'flex', 'flexDirection':'row', 'justifyContent':'center'}), 
                html.Div(children = [dcc.Dropdown(newVariableList, newVariableList[0], multi = True, id = 'dropVars', className = 'drop-var', style = {'position':'relative', 'fontSize': '15px'})], 
                         style = {'alignItems':'start'})], 
                    style =  {'position':'relative', 'width':'30%', 'marginLeft':'1%'}),
            
            
            dcc.RadioItems(['Raw', 'Normalized'], 'Raw', id = 'dtype', style = {'position':'relative','top': '30%', 'fontSize': '20px', 'marginLeft': '1%', }), 
            dcc.Checklist(['Add graph?'], id = 'graphBool', style = {'position':'relative','top': '30%', 'fontSize': '20px', 'marginLeft': '1%', }), 
        ], style = {'position':'relative', 'width':'100%', 'marginTop':'1%', 'display':'flex', 'flexDirection':'row'}), 
    ], style = {'marginLeft': '6%', 'display':'flex', 'flexDirection': 'row', 'height':'18%', 'marginTop':'5%', 'width':'90%'}),
    
    html.Div(className = 'results', children = [
        html.Div(id = 'errorMessage', children = [
            html.H1('Select a Region')
        ], style = {'position':'relative', 'top':'30vh', 'textAlign':'center', 'alignItems':'center', 'display': 'block', 'fontSize':'30px', 'marginLeft':'40%'}), # 'position':'absolute', 'textAlign':'center', 'marginTop': '50%', 'display': 'block', 'fontSize':30}

        html.Div(id = 'metricTable', className = 'table-holder', children = [
            dummy_table, 
        ], style = {'marginLeft': '5%', 'marginTop':'2%'}), #'marginLeft': '5%', 'justifyContent': 'center', 'marginTop':'2%'

        html.Div(id = 'analytics', children = [
            html.Div(id = 'scatterHolder', children = [
                html.Div(id = 'regHolder', children = [
                    html.Div(children = 'y = ', style = {'fontSize':'30px'}, id = 'lineDisplay'),
                    html.Div(children = 'R-squared:' , style = {'fontSize':'30px', 'marginLeft':'3%'}, id = 'r2display'), 
                ], style = {'display':'flex', 'flexDirection': 'row'}), 
                dcc.Graph(figure = dummy_graph, id = 'scatterPlot')
            ], style = {'positon':'relative', 'marginLeft':'1%', 'marginTop':'5%'}), 

            html.Div(id = 'scatterOpts', children = [
                html.H3('X-Axis variable'), # choose the x axis variable
                dcc.Dropdown(newVariableList, newVariableList[0], id = 'xdat', style = {'width':'200px'}),
                html.H3('Y-Axis variable'), # choose the y axis variable
                dcc.Dropdown(newVariableList, newVariableList[1], id = 'ydat', style = {'width':'200px'})
            ], style = {'display':'flex', 'flexDirection':'column', 'marginTop':'10%', 'marginLeft':'2%'}), # 'display':'flex', 'flexDirection':'column', 'marginTop':'30vh'} 
        ], style = {'position':'relative', 'marginLeft':'48%', 'display': 'flex', 'flexDirection': 'row', 'display':'none', 'marginTop':'5%'}), #'position':'relative', 'marginLeft':'2%', 'display': 'flex', 'flexDirection': 'row', 'display':'none'}
    ], style = {'display':'flex', 'overflow':'auto', 'marginTop': '15%', 'marginLeft':'6%', 'width':'90%', 'height': '70%'}), #'position':'relative', 'marginLeft':'2%'

    # html.Div(className = 'cright', children = 'LLNL CODE - 850907', style = {'marginTop': 100 , 'marginLeft':100}),

    dcc.Store(id = 'previous_click-store'),
    dcc.Store(data = modelListLoca, id = 'model-list-loca2'),
    dcc.Store(data = modelListStar, id = 'model-list-star'),
    dcc.Store(data = choro_dict, id = 'choroDict'), 
    dcc.Store(id = 'storedParams'), 
    
    ## graph stores
    dcc.Store(data = {'xData': [], 'yData': []}, id = 'xyStore'),
    dcc.Store(data = {}, id = 'regStore'),
    # dcc.Store(id = 'r2Store'), 
    
    ## previous call data
    dcc.Store(data = {key:'' for key in ['regionName', 'method', 'comparison', 'metric', 'variables', 'dtype']}, id = 'oldParams'),
    dcc.Store(data = {'median': {}, 'iqr': {}}, id = 'statStore')
    
], style = {'display': 'flex', 'flexDirection': 'column'})

## removing unneeded dropdowns if the view type is "Method"

In [53]:
app.clientside_callback(
    """
    /* JS code
        If we are comparing methods instead of variables, remove the option to choose comparison datasets by returning display:none in style option
    */ 
    
    function(params, old_style) { 
    
        style = JSON.parse(JSON.stringify(old_style))

        if (params['view'] == 'Method') { 
            style['display'] = 'none';
            return style; 
        } else { 
            style['display'] = 'block';
            return style;
        }
    }
    
    """, 
    Output(component_id = 'compHolder', component_property = 'style'), 
    Input(component_id  = 'storedParams', component_property = 'data'), 
    State(component_id  = 'compHolder', component_property = 'style'),
    preventinitial_call = True)

In [54]:
app.clientside_callback(
    """
    function(params, old_style) { 
        /* JS code
            If we are comparing methods instead of variables, remove the option to choose method by returning display:none in style option
        */ 
    
        style = JSON.parse(JSON.stringify(old_style))

        if (params['view'] == 'Method') { 
            style['display'] = 'none';
            return style; 
        } else { 
            style['display'] = 'block';
            return style;
        }
    }
    
    """, 
    Output(component_id = 'methodHolder', component_property = 'style'), 
    Input(component_id  = 'storedParams', component_property = 'data'), 
    State(component_id  = 'methodHolder', component_property = 'style'),
    prevent_initial_call = True)

In [55]:
app.clientside_callback(
    """
    function(params) { 
        /* JS code
            If we are comparing methods instead of variables, remove the option to choose variables by returning display:none in style option
        */ 
        if (params['view'] == 'Method') {
            return false;
        } else { 
            return true; 
        }
    }
    """, 
    Output(component_id = 'dropVars', component_property = 'multi'), 
    Input(component_id  = 'storedParams', component_property = 'data'), 
    prevent_initial_call = True)

## constraining the comparison datasets

In [56]:
app.clientside_callback( 
    """ 
    function(method) { 
        
        /* JS code
            return the correct list of option for comparison dataset depending on which method is selected.
            LOCA2: ['PRISM', 'Livneh']
            STAR:  ['PRISM', 'NClimGrid'] 
        */
    
        if (method == 'loca2') {
            return ['PRISM', 'Livneh']; 
        } else if (method == 'star') {
            return ['PRISM', 'NClimGrid'];
        }
    }
    """, 
    Output(component_id = 'dropComp', component_property = 'options'), 
    Input(component_id = 'dropMethod', component_property = 'value'), 
    prevent_initial_call = False)

# select the first value to automatically display if the options change
app.clientside_callback(
    """
    function(options) { 
        return options[0];
    }
    """, 
    Output(component_id = 'dropComp', component_property = 'value'), 
    Input(component_id = 'dropComp', component_property = 'options'),
    prevent_initial_call = False)

## constraining metrics

In [57]:
# app.clientside_callback(
#     """ 
#     function(comp) {
        
#         if (comp == 'Livneh' || comp == 'NClimGrid') {
#             return ['rms', 'rmsc', 'mae', 'bias', 'corr']; 
#         } else { 
#             return ['rms', 'rmsc', 'mae', 'bias', 'corr']; 
#         } 
#     }
#     """, 
    
#     Output(component_id = 'dropMetric', component_property = 'options'), 
#     Input(component_id  = 'dropComp', component_property = 'value'),
#     prevent_initial_call = False)

# select the first value to automatically display if the options change

app.clientside_callback(
    """
    /* JS code
        select the first value to automatically display if the options change
    */ 
    
    function(options, oldParams) { 
        let params = JSON.parse(JSON.stringify(oldParams));
        let oldMetric = params['metric']; 
        
        if (options.includes(oldMetric)) {
            return oldMetric
        } else {
            return options[0];
        }
    }
    """, 
    Output(component_id = 'dropMetric', component_property = 'value'), 
    Input(component_id = 'dropMetric', component_property = 'options'), 
    State(component_id = 'oldParams', component_property = 'data'), 
    prevent_initial_call = False)


## Updating the parameter inputs

In [58]:
app.clientside_callback(
    """
    function(region, method, metric, comp, vars, dtype, view) { 
    
        /* JS code
            This is the dictionary that defines the parameters of the dashboard display. Rather than using all of these individual parameters as inputs 
            later on in the code, these stored parameters will be the single input to later functions. 
        */ 
        
        
        let params = {};
        
        if (typeof region == 'undefined') {
            region = 'No Region Selected'; // Error option, in this case a "select a region" message will be displayed
        }
        
        params['regionName'] = region; 
        params['method'] = method;
        params['metric'] = metric;
        params['comparison'] = comp;
        params['variables'] = vars; 
        params['dtype'] = dtype;
        params['view']  = view; 
                
        return params;
    }
        
    """,
    Output(component_id = 'storedParams', component_property = 'data'), 
    Input(component_id = 'previous_click-store', component_property = 'data'), 
    Input(component_id   = 'dropMethod', component_property = 'value'),
    Input(component_id   = 'dropMetric', component_property = 'value'), 
    Input(component_id   = 'dropComp', component_property = 'value'), 
    Input(component_id   = 'dropVars', component_property = 'value'), 
    Input(component_id   = 'dtype', component_property = 'value'), 
    Input(component_id   = 'dropView', component_property = 'value'), 
    prevent_initial_call = True)

In [59]:
app.clientside_callback(
    """
    function(params, old_style) {   
    
        /* JS code
            If there is no region selected in our params input, return the error message
        */
        
        let style = JSON.parse(JSON.stringify(old_style)); 
        
        if (params['regionName'] === 'No Region Selected') {
            style['display'] = 'block';
            return style;
        } else {
            style['display'] = 'none';
            return style;
        }
    }
    """, 
    Output(component_id = 'errorMessage', component_property = 'style'), 
    Input(component_id  = 'storedParams', component_property = 'data'),
    State(component_id  = 'errorMessage', component_property = 'style'), 
    prevent_initial_call = True)

## showing analytics graph

In [60]:
app.clientside_callback(
    """ 
    function(graphBool, style_input, old_style) { 
    
        /* JS code
            If the user wants to add a graph, set the display style to "block" to make the graph visible
        */ 
        
        let style_values = Object.values(style_input);
        let style        = JSON.parse(JSON.stringify(old_style)); // get the old style parameters 
        
        if ((graphBool != 'Add graph?') | (style_values.includes('block'))) {
            style['display'] = 'none';
            return style;
            // return {'display': 'none'}
        } else {
            style['display'] = 'flex';
            return style; 
            // return {'display': 'flex', 'flexDirection': 'row'}
        }
    }
    """, 
    Output(component_id = 'analytics', component_property = 'style'), 
    Input(component_id  = 'graphBool', component_property = 'value'), 
    Input(component_id  = 'errorMessage', component_property = 'style'),
    State(component_id  = 'analytics', component_property = 'style'), 

    prevent_initial_call = True)

## Updating the stored region

In [61]:
## for ...
app.clientside_callback(
    """
    
    /* JS code
        Store the region name that the user selected last. 
    */ 
    
    function(click, type) {
        
        if (window.dash_clientside.callback_context.triggered[0]['prop_id'].includes('dropType')) { 
            return 'No Region Selected';
        }
        
        return click.points[0].customdata[0];
    }
    """,
    Output(component_id = 'previous_click-store', component_property = 'data'),
    Input(component_id  = 'usmap',        component_property = 'clickData'), 
    Input(component_id  = 'dropType', component_property = 'value'), 
    prevent_initial_call = True)

## Changing and coloring the map

In [62]:
# dealing with choropleth map

app.clientside_callback(
    """ 
    function(regionSelect, subregion, choro_dict) {
    
        /* JS code
            Input regionType    -> Return the appropriate choropleth map of the CUS with boundaries according to the user selection
            Input previousClick -> Add a column signifying the users selection. Color the regions within the map based on this column.
        */ 
        let defaultChoro;
        
        if (regionSelect === 'nca4') { 
            defaultChoro = JSON.parse(JSON.stringify(choro_dict['NCA4 Region'])); 
        } else if (regionSelect == 'states') { 
            defaultChoro = JSON.parse(JSON.stringify(choro_dict['States']))
        } else if (regionSelect == 'cus') { 
            defaultChoro = JSON.parse(JSON.stringify(choro_dict['cus']))
        } else {
            defaultChoro = JSON.parse(JSON.stringify(choro_dict['Half']))
        }
        
        if (window.dash_clientside.callback_context.triggered[0].prop_id.includes('regionSelect')) { // Exit early; don't need a new column for selected region
            return defaultChoro;
        }
        
        defaultChoro.data[0].colorscale[0] = [0, 'powderblue']; 
        defaultChoro.data[0].colorscale[1] = [1, 'royalblue']; 
        
        // 1 is not selected
        // 0 is selected
        
        let newZ = [];
        let regionNames = [...defaultChoro.data[0].customdata]; 
        
        for (const element of regionNames) {
           if (element[0] === subregion) {
               newZ.push(1); 
            } else {
                newZ.push(0); 
            }
        }
        
        defaultChoro.data[0].z = newZ; 
        
        return defaultChoro
    }
    """, 
    Output(component_id='usmap', component_property='figure'), 
    Input(component_id ='dropType', component_property='value'), 
    Input(component_id ='previous_click-store', component_property = 'data'),
    State(component_id ='choroDict', component_property='data'), 
    prevent_initial_call = True)

## updating columns

In [63]:
# //, regionName, variables) {
app.clientside_callback(
    """
    function(parameters) {
        /* JS code
            Updates the columns of the visible table with the variable names (Variable view) or the method - comparison dataset options (method view)
        */ 
        
        let view      = parameters['view'];
        // console.logview);
        let variables = parameters['variables']; 
        let regionName = parameters['regionName'];
        
        if (regionName === 'No Region Selected') {
            return [];
        }
        let columns = new Array();
        
        if (view == 'Method') {
            // console.log'here')
            
            columns = columns.concat({'name':'Models', 'id':'Models'})
            
            for (let columnName of ['LOCA2-PRISM', 'LOCA2-Livneh', 'STAR-PRISM', 'STAR-NClimGrid']) {
                columns = columns.concat({'name':columnName, 'id':columnName});
            }
            // console.logcolumns);
            return columns;
        }
        
        let variableList;
        
        if (typeof variables === 'string') {
            variableList = [variables];
        } else { 
            variableList = variables;
        }
                
        columns = columns.concat({'name': 'Models', 'id' : 'Models'})
        
        for (let variable of variableList) {
            columns = columns.concat({'name': variable, 'id' : variable}) // Format for dashTable column entries {Name:ColumnName, ID: ColumnID}
        }        
        
        return columns;
    }
    """, 
    Output(component_id    = 'tbl', component_property = 'columns'), 
    Input(component_id     = 'storedParams', component_property = 'data'),
    # Input(component_id   = 'button', component_property = 'n_clicks'),
    # State(component_id   = 'previous_click-store', component_property = 'data'), 
    # State(component_id   = 'dropVars', component_property = 'value'),
    # State(component_id   = 'oldVars', component_proeprty  = 'value'), 

    prevent_initial_call = True)

## get Data 

In [64]:
app.clientside_callback(
    """
    // function(cols, regionName, method, metric, type, comp, variableList, dtype, modelListLoca, modelListStar, oldVars, oldMethod, oldComp, oldMetric, oldRegion, oldNorm, oldData) {
    
    function(cols, params, type, modelListLoca, modelListStar, oldParams, oldStats, oldData) {
        
        /* JS code
            The primary dashboard function that reads the data from the /assets/fullFiles/ folder. 
            
            Input ->  Dashtable Columns (ONLY update the table if the columns have changed)
            Output -> Array in the form of a DashTable.columns object
        */ 
        
        
        if (cols.length == 0) { // If we have no columns to get data for
            return [];
        }
        
        let columnNames = new Array(); 
        
        for (let columnObj of cols) {            
            if (columnObj['name'] != 'Models') { // Skipping the first column named 'Models'
                columnNames = columnNames.concat(columnObj['name']);
            }
        }
        
        // Getting parameters and previous dashboard state parameters
        
        let regionName    = params['regionName'];
        let method        = params['method'];
        let metric        = params['metric']; 
        let comp          = params['comparison']; 
        let dtype         = params['dtype'];
        let variableList  = params['variables'];
        
        
        let oldRegion     = oldParams['regionName'];
        let oldMethod     = oldParams['method'];
        let oldMetric     = oldParams['metric']; 
        let oldComp       = oldParams['comparison']; 
        let oldNorm       = oldParams['dtype'];
        let oldVars       = oldParams['variables'];
        let oldView       = oldParams['view']
        
        let boolOld; // To save time, we can reuse the data that we've retrieved previously if the model list hasn't changed.        
        
        if (oldMethod == method && oldComp == comp && oldMetric == metric && oldRegion == regionName && oldNorm == dtype) { // only changing the list of variables
            boolOld = true; 
        } else if (oldMethod == method && oldComp == comp && oldMetric == metric && oldRegion == regionName && !(oldNorm == dtype)) {
            boolOld = false;
        } else { 
            boolOld = false; 
        }
        
        console.log(boolOld);
        
        
        // I'm not sure modelList is correctly defined when the dashboard view is 'Method'
        
        let dVariableList;
        let modelList = (method =='loca2') ? modelListLoca : modelListStar; // (conditional) ? TrueValue : FalseValue
        
        
        
        if (params['view'] == 'Method') { // In this case, dVariableList = ['LOCA2-PRISM', ...]
            dVariableList = columnNames; 
            
        } else {
            if (typeof variableList === 'string') { // Only one variable
                dVariableList = [variableList];
            } else { 
                dVariableList = variableList;       // List of variables
            }
        }

        let testArr = new Array(); 
        
        async function getData(models, variables, bool, oldData, paramVariable, paramView) { //  async function that retrieves the data from assets/fullFiles
            
            /*
                models        -> List of models depending on LOCA or STAR dataset
                variables     -> variable list from the parameter input
                bool          -> whether or not we have old data ( Model list wasn't changed )
                oldData       -> old data to draw from
                paramVariable -> If the dashboard view is 'method', only 1 variable will be given
                paramView     -> Dashboard view state ('method' or 'variable'). Usefull for getting the correct file name
            */ 
            
            
        
            ind = 0         // dummy counter, will count the index we are currently looping through for [modelObj 1, modelObj 2, ...]
            let nDict = {}; // "normalized" data dictionary
            
            for ( let model of models ) {
                let modelObj; 
                
                if (bool) { 
                    modelObj = oldData[ind]; // Each entry in modelObj array is a dictionary of the form {'Models': ModelName}
                } else {
                    modelObj = {'Models' : model}; 
                }
                 
                for (let variable of variables) {
                
                    let qAdd = '';
                    let sAdd = '';
                    
                    let variable_name; 
                    
                    if (paramView == 'Method') { 
                        variable_name = paramVariable;
                    } else { 
                        variable_name = variable;
                    }
                    
                    // Getting the correct variable name if the variable is seasonal or quantile
                    
                    if (variable_name.includes('q')) {
                        qAdd = '.' + variable_name.split('_')[0];
                        variable_name = variable_name.split('_').slice(-1);
                    }
                    
                    if ( (variable_name.includes('DJF')) | (variable_name.includes('MAM')) | (variable_name.includes('JJA')) | (variable_name.includes('SON')) ) {
                        sAdd = '.' + variable_name.split('_').slice(-1);
                        
                        // variable_name = 'seasonal' + (variable.split('_').slice(0 , -1)).join('_');
                        
                        if (! variable_name.startsWith('mean')) { 
                            variable_name = 'seasonal_' + variable_name.split('_').slice(0,-1).join('_')
                        } else { 
                            variable_name = 'seasonal' + variable_name.split('_').slice(0,-1).join('_')
                        }
                    }
                    
                    if ((variable in modelObj) && (paramView != 'Method')) { // If we already have the data (i.e. we only added a new variable and changed no other dashboard parameters)
                        continue
                    }
                    
                    let file_name;
                    
                    if (paramView == 'Method') { // Right now our "variables" are LOCA2-PRISM, STAR-PRISM, etc. Splicing to get the method and comparison dataset
                        let nameComps = variable.split('-');
                        let newMethod = nameComps[0].toLowerCase();
                        let newComp   = nameComps[1].toLowerCase(); 
                        
                        file_name = 'assets/fullFiles/' + [newMethod, newComp, variable_name, model, type].join('.') + qAdd + sAdd + '_stats.json' // file name convention
                    } else { 
                        file_name = 'assets/fullFiles/' + [method, comp.toLowerCase(), variable_name, model, type].join('.') + qAdd + sAdd + '_stats.json'
                    }
                    
                    console.log(file_name);
                    
                    let key = [regionName, metric].join('.'); // Also convention, the JSON files in fullFiles are dictionaries containing statistics, where the keys are regionName.metric
                    
                    try { 
                        response = await fetch(file_name);
                        response = await response.json();
                        data = await response[key];
                        paramValue     = await parseFloat(data.toPrecision(3))
                    } catch (error) { 
                        // console.log(error)
                        paramValue = null
                    }
                    
                    modelObj[variable]   = await paramValue;
                    
                    if (variable in nDict) { // If we've already computed the normalized value ("in" checks the list of keys if passed a dictionary)
                        nDict[variable] = await nDict[variable].concat(paramValue);
                        
                    } else {                 // Will do the appropriate calculation later using the data from this dictionary
                        nDict[variable] = await [paramValue];
                    }
                }
                ind++;
                testArr = testArr.concat(modelObj)
            }
            
            if (dtype == 'Raw') { // && (!boolN)
                return testArr; 
            }
            
            function median(arr) { 
            
                // Calculate the median of an array 
                
                const sortArr = arr.slice().sort((a,b) => a-b); 
                const mid = Math.floor(sortArr.length/2); 
                
                if (arr.length % 2 === 0 ) { 
                    return (sortArr[mid-1] + sortArr[mid])/2; 
                } else { 
                    return sortArr[mid]; 
                }
            }
            
            function iqr(arr) { 
                // Calculate the IQR of an array
                
                const sortArr = arr.slice().sort((a,b) => a-b); 
                const mid = Math.floor(sortArr.length/2); 
                const q25 = median(sortArr.slice(0,mid))
                const q75 = median(sortArr.slice(mid + (sortArr.length % 2))); 
                
                return q75 - q25;
            }

            let keys = Object.keys(nDict); 
            
            // if (!boolN) {
                //console.log('no old data, calculating normalized data');
                
            for (let key of keys) { // Replacing the value within the DashTable object to (value - median) / IQR. Floating for readability
                let varMedian = median(nDict[key])
                let varIQR    = iqr(nDict[key])

                for (var i = 0; i < testArr.length; i++) {
                    if (testArr[i][key] == null) { 
                        testArr[i][key] = null; 
                    } else {
                        testArr[i][key] = parseFloat(((testArr[i][key] - varMedian)/varIQR).toPrecision(3));
                    }
                }
            }
            return testArr;
                
            //} else {
                //console.log('old data, calculating normalized data');
                //for (let key of keys) {
                //    let varMedian = oldStats['median'][key]
                //    let varIQR    = oldStats['iqr'][key]
                
                //   for (var i = 0; i < testArr.length; i++) { 
                //        if (dtype == 'Norm') {
                //            testArr[i][key] = parseFloat(((testArr[i][key] - varMedian)/varIQR).toPrecision(3));
                //        } else { 
                //            testArr[i][key] = parseFloat((testArr[key]*varIQR + varMedian).toPrecision(3));
                //        }
                //    }
                // }
                // return testArr; 
            //}
        }
        
        return getData(modelList, dVariableList, boolOld, oldData, params['variables'], params['view']);
    }
    """, 
    Output(component_id  = 'tbl', component_property = 'data'),  
    Input(component_id   = 'tbl', component_property = 'columns'), 
    State(component_id   = 'storedParams', component_property = 'data'), 
    State(component_id   = 'dropType',   component_property = 'value'), 
    State(component_id   = 'model-list-loca2', component_property = 'data'), 
    State(component_id   = 'model-list-star', component_property = 'data'), 
    State(component_id   = 'oldParams', component_property = 'data'),
    State(component_id   = 'statStore', component_property = 'data'), 
    State(component_id   = 'tbl', component_property = 'data'), 

    
    prevent_initial_call = True)

In [65]:
# """
#      |  - style_data_conditional (list of dicts; optional):
#      |      Conditional CSS styles for the data cells. This can be used to
#      |      apply styles to data cells on a per-column basis.
#      |  
#      |      `style_data_conditional` is a list of dicts with keys:
#      |  
#      |      - if (dict; optional)
#      |  
#      |          `if` is a dict with keys:
#      |  
#      |          - column_editable (boolean; optional)
#      |  
#      |          - column_id (string | list of strings; optional)
#      |  
#      |          - column_type (a value equal to: 'any', 'numeric', 'text', 'datetime'; optional)
#      |  
#      |          - filter_query (string; optional)
#      |  
#      |          - row_index (number | a value equal to: 'odd', 'even' | list of numbers; optional)
#      |  
#      |          - state (a value equal to: 'active', 'selected'; optional)
# """

## coloring table if normalized

In [66]:
app.clientside_callback(
    """
    function(data, cols, params) { 
    
        /* JS code
            Function that only runs if normalized data is requested. Modifies the "style data conditional" property of the DashTable
            
        */
    
        let variables = params['variables']; 
        let dtype = params['dtype']
        let view  = params['view']
        
        if (dtype == 'Raw') { 
            return; // Get out
        }
        
        // getting column names for "method" view
        
        let columnNames = new Array(); 
        
        for (let columnObj of cols) {
            if (columnObj['name'] != 'Models') {
                columnNames = columnNames.concat(columnObj['name']);
            }
        }
        
        function median(arr) { 
            const sortArr = arr.slice().sort((a,b) => a-b); 
            const mid = Math.floor(sortArr.length/2); 

            if (arr.length % 2 === 0 ) { 
                return (sortArr[mid-1] + sortArr[mid])/2; 
            } else { 
                return sortArr[mid]; 
            }
        }

        function lowerUpper(arr) { 
            const sortArr = arr.slice().sort((a,b) => a-b); 
            const mid = Math.floor(sortArr.length/2); 
            const q25 = median(sortArr.slice(0,mid))
            const q75 = median(sortArr.slice(mid + (sortArr.length % 2))); 

            return {'q25': q25, 'q75': q75}; 
        }
    
        let dVariableList;
        
        if (view == 'Method') {
            dVariableList = columnNames; 
        } else {
            if (typeof variables === 'string') {
                dVariableList = [variables];
            } else { 
                dVariableList = variables;
            }
        }
        
        let style_arr = []; 
        
        for (let variable of dVariableList) { 
            let variableVals = []; 
            
            for (let row of data) {
                variableVals.push(row[variable]) // Get the data that we will need to determine the coloring pattern
            }
            
            normMedian = median(variableVals);        // median 
            normQ25 = lowerUpper(variableVals)['q25'] // lower quartile
            normQ75 = lowerUpper(variableVals)['q75'] // upper quartile
            
            // dark blue   0% -> 25%
            // light blue 25% -> 50%
            // light red  50% -> 75% 
            // dark red   75% -> 100%
            
            let boundsDict = {'#5656FA': [Math.min(...variableVals), normQ25], 
                              '#C3C3F6': [normQ25, normMedian], 
                              '#F6CDC3': [normMedian, normQ75], 
                              '#E53F1B': [normQ75, Math.max(...variableVals)+0.5],
                              '#FFFFFF': null}
                              
            for (let [color, bound_arr] of Object.entries(boundsDict)) { // loops through key, [min, max] threshold array
                if (color == '#FFFFFF') { // handle NaN entries by coloring white
                    style_obj = {'if' : {'filter_query': `{${variable}} is nil`, 
                         'column_id': `${variable}`
                         }, 
                        'background_color': `${color}`
                    }
                } else { // append style conditional statement
                    style_obj = {'if' : {'filter_query': `{${variable}} >= ${bound_arr[0]} && {${variable}} < ${bound_arr[1]}`, 
                                         'column_id': `${variable}`
                                         }, 
                                        'background_color': `${color}`}
                }
                            
            style_arr.push(style_obj); 
            }
        }
        return style_arr;
    }
    """, 
    Output(component_id = 'tbl', component_property = 'style_data_conditional'),
    Input(component_id  = 'tbl', component_property = 'data'), 
    State(component_id  = 'tbl', component_property = 'columns'), 
    State(component_id  = 'storedParams', component_property = 'data'), 
    prevent_initial_call = True)

In [67]:
# template for reference
"""
tooltip_data=[
    {column: {'value': f'![](assets/images/{getImageURL(region, model_name, column, comparison_ds, False)})', 'type': 'markdown'}
        for column, value in row.items()
    } for row, model_name in zip(ttip_data, index)
]
"""


"\ntooltip_data=[\n    {column: {'value': f'![](assets/images/{getImageURL(region, model_name, column, comparison_ds, False)})', 'type': 'markdown'}\n        for column, value in row.items()\n    } for row, model_name in zip(ttip_data, index)\n]\n"

In [68]:
app.clientside_callback(
    """
    function(data, cols, variables, method, comp,regionType, regionName, params) {
        
        /* JS code
                Link the appropriate thumbnail image (assets/images_new) to the tooltip_data property of the DashTable
        */ 
        
        //let dVariableList; 
        
        if (cols.length == 0) { 
            return [];
        }
        
        let columnNames = new Array(); 
        
        for (let columnObj of cols) {            
            if (columnObj['name'] != 'Models') {
                columnNames = columnNames.concat(columnObj['name']);
            }
        }
        
        let dVariableList;
        
        if (params['view'] == 'Method') {
            dVariableList = columnNames; 
            
        } else {
            if (typeof variables=== 'string') {
                dVariableList = [variables];
            } else { 
                dVariableList = variables;
            }
        }
        
        let tooltip_arr = []; 
        
        for (let row of data) { // row = {'Models': 'ModelName', variableName 1: variableValue 1, ...}
            let model_name = row['Models'];
            let tooltip_dict = {};

            for (let variable of dVariableList) {                  
                let sAdd = ''; 
                let qAdd = ''; 
                
                let variable_name; 
                
                if (params['view'] == 'Method') { 
                    variable_name = variables; 
                } else { 
                    variable_name = variable; 
                }
                // getting the appropriate season/quantile add-on string
                
                if ( (variable_name.includes('DJF')) | (variable_name.includes('MAM')) | (variable_name.includes('JJA')) | (variable_name.includes('SON')) ) {
                    sAdd = '.' + variable_name.split('_').slice(-1);
                    
                    if (! variable_name.startsWith('mean')) { 
                        variable_name = 'seasonal_' + variable_name.split('_').slice(0,-1).join('_')
                    } else { 
                        variable_name = 'seasonal' + variable_name.split('_').slice(0,-1).join('_')
                    }
                }
                
                if (variable_name.includes('q')) {
                    qAdd = '.' + variable_name.split('_')[0];
                    variable_name = variable_name.split('_').slice(-1);
                }
                
                if (params['view'] == 'Method') { // file naming
                    let newMethod = variable.split('-')[0].toLowerCase(); 
                    let newComp   = variable.split('-')[1].toLowerCase(); 
                    tooltip_dict[variable] =  {'value': `![](assets/images_new/${regionName.replace(/ /g, '')}/${newMethod}.${newComp.toLowerCase()}.${variable_name}.${model_name}.${regionType}${qAdd}${sAdd}.${regionName.replace(/ /g, '')}.jpg)`, 'type': 'markdown'}
                } else {
                    tooltip_dict[variable] =  {'value': `![](assets/images_new/${regionName.replace(/ /g, '')}/${method}.${comp.toLowerCase()}.${variable_name}.${model_name}.${regionType}${qAdd}${sAdd}.${regionName.replace(/ /g, '')}.jpg)`, 'type': 'markdown'}
                }
            }
            
            tooltip_arr.push(tooltip_dict)
        }        
        return tooltip_arr; 
    } 
    """, 
    Output(component_id = 'tbl', component_property = 'tooltip_data'), 
    Input(component_id  = 'tbl', component_property = 'data'),
    State(component_id  = 'tbl', component_property = 'columns'), 
    State(component_id  = 'dropVars', component_property = 'value'), 
    State(component_id  = 'dropMethod', component_property = 'value'), 
    State(component_id  = 'dropComp', component_property = 'value'), 
    State(component_id  = 'dropType', component_property = 'value'), 
    State(component_id  = 'previous_click-store', component_property = 'data'),
    State(component_id  = 'storedParams', component_property = 'data'), 
    
    prevent_initial_call = True)

# Storing the old parameters

In [69]:
app.clientside_callback(
    """
    function(tableUpdate, regionName, method, metric, comp, variables, dtype, view) {
    
        /* JS code
            Storing the most recent parameters so that we can use their values in the next dashboard iteration
        */ 
        
        let params = {};
        
        params['regionName'] = regionName; 
        params['method'] = method;
        params['metric'] = metric;
        params['comparison'] = comp;
        params['variables'] = variables; 
        params['dtype'] = dtype;
        params['view']  = view;
                
        return params;
    }
    """, 
    
    Output(component_id = 'oldParams', component_property = 'data'),
    
    Input(component_id   = 'tbl', component_property = 'data'), 
    State(component_id   = 'previous_click-store', component_property = 'data'), 
    State(component_id   = 'dropMethod', component_property = 'value'),
    State(component_id   = 'dropMetric', component_property = 'value'), 
    State(component_id   = 'dropComp', component_property = 'value'), 
    State(component_id   = 'dropVars', component_property = 'value'), 
    State(component_id   = 'dtype', component_property = 'value'), 
    State(component_id   = 'dropView', component_property = 'value'), 
    prevent_initial_call = True)
    

In [70]:
# ## storing the old median/iqr

# app.clientside_callback(
#     """
#     function(data, variableList, oldStats, dtype) { 
#         /*
#             Not used currently. Idea was to save time if there was no need to recalculate the median/iqr values for each variable. Doesn't save much time and didn't work
#         */
        
        
#         if (dtype == 'Normalized') { 
#             return oldStats;
#         }
        

    
#         // console.log(data);
#         // median/iqr functions
        
#         let dVariableList;
        
#         if (typeof variableList === 'string') {
#             dVariableList = [variableList];
#         } else { 
#             dVariableList = variableList;
#         }
    
#         function median(arr) { 
#             const sortArr = arr.slice().sort((a,b) => a-b); 
#             const mid = Math.floor(sortArr.length/2); 

#             if (arr.length % 2 === 0 ) { 
#                 return (sortArr[mid-1] + sortArr[mid])/2; 
#             } else { 
#                 return sortArr[mid]; 
#             }
#         }

#         function iqr(arr) { 
#             const sortArr = arr.slice().sort((a,b) => a-b); 
#             const mid = Math.floor(sortArr.length/2); 
#             const q25 = median(sortArr.slice(0,mid))
#             const q75 = median(sortArr.slice(mid + (sortArr.length % 2))); 

#             return q75 - q25;
#         }
        
#         let iqrDict    = {};
#         let medianDict = {};
        
#         for (let variable of dVariableList) {
            
#             let arr = [];
            
#             for (let row of data) {
#                 arr.push(row[variable])
#             }
            
#             medianDict[variable] = median(arr);
#             iqrDict[variable]    = iqr(arr);
#         }
        
#         return {'median':medianDict, 'iqr': iqrDict}
    
#     }
#     """,
#     Output(component_id = 'statStore', component_property = 'data'), 
#     Input(component_id  = 'tbl', component_property = 'data'),
#     State(component_id  = 'dropVars', component_property = 'value'),
#     State(component_id  = 'statStore', component_property = 'data'), 
#     State(component_id  = 'dtype', component_property = 'value'),
#     prevent_initial_call = True);
    

In [71]:
app.clientside_callback(
    """
    function(columns) {
        
        /* JS code
            Updating X-Variable options to the chosen variables for use in the analytics graph
        */ 
    
        let arr = []; 
        
        for (let columnObj of columns) {
            if (columnObj['name'] != 'Models') {
                arr.push(columnObj['name'])
            }
        }
        return arr;
    }
    """, 
    
    Output(component_id = 'xdat', component_property = 'options'), 
    Input(component_id = 'tbl', component_property = 'columns'), 
    prevent_initial_call = True)

app.clientside_callback(
    """
    function(options, value) { 
        /* JS code
            Updating X-Variable display value to the first variable option
        */ 
        return value;
    }
    """, 
    
    Output(component_id = 'xdat', component_property = 'value'), 
    Input(component_id   = 'xdat', component_property = 'options'), 
    State(component_id   = 'xdat', component_property = 'value'),
    prevent_initial_call = True)
    


app.clientside_callback(
    """
    function(options) {
        /* JS code
            Updating Y-Variable options to the chosen variables for use in the analytics graph (same as x-data options)
        */ 
        return options;
    }
    """, 
    
    Output(component_id = 'ydat', component_property = 'options'), 
    Input(component_id= 'xdat', component_property = 'options'), 
    prevent_initial_call = True)
    
app.clientside_callback(
    """
    function(options, value) { 
        /* JS code
            Updating Y-Variable display value to the first variable option
        */ 
        return value;
    }
    """, 
    
    Output(component_id = 'ydat', component_property = 'value'), 
    Input(component_id   = 'ydat', component_property = 'options'), 
    State(component_id   = 'ydat', component_property = 'value'), 
    prevent_initial_call = True)

In [72]:
app.clientside_callback(
    """
    function(xVar, yVar, tblData, graph, xyStore) {
    
        /* JS Code 
            Get the 1D variable data from the tableData store based on the selected XY variable options. 
            Store the data
            
            We are storing the data rather than directly plotting it because we need to use it for creating three other outputs: the plot, correlation coeff display, and slope/intercept display. 
        
        */ 
        
        let graphBool = (graph == 'Add graph?');
        
        const average = array => array.reduce((a, b) => a + b) / array.length 
        
        newXY = JSON.parse(JSON.stringify(xyStore)) // This format is a useful way to create a copy of a variable that doesn't modify the original 

        if (!graphBool) {  
            return newXY
        }
        
        let xdata = [];
        let ydata = [];
        let hdata = [];

        for ( let row of tblData ) {
            xdata.push(row[xVar]);
            ydata.push(row[yVar]);
            hdata.push(row['Models']);
        }

        newXY['xData'] = xdata;
        newXY['yData'] = ydata;
        newXY['hoverData'] = hdata;
        
        return newXY;
    }
    """,
    
    Output(component_id = 'xyStore', component_property = 'data'), 
    Input(component_id  = 'xdat', component_property = 'value'), 
    Input(component_id  = 'ydat', component_property = 'value'),
    Input(component_id  = 'tbl', component_property ='data'), 
    State(component_id  = 'graphBool', component_property = 'value'), 
    State(component_id  = 'xyStore', component_property = 'data'),
    # State(component_id  = '
    prevent_initial_call = True)

In [73]:
app.clientside_callback(
    """
    function(regStore, xyStore, graph, scatterPlot, xVar, yVar) {
    
        /* JS Code 
            Plot the stored XY data and a least-squares line of best fit
        */ 
        
        let graphBool = (graph == 'Add graph?');
        
        const average = array => array.reduce((a, b) => a + b) / array.length // mean function definition 
        
        scatterCopy = JSON.parse(JSON.stringify(scatterPlot));

        xyData = JSON.parse(JSON.stringify(xyStore));
        regData= JSON.parse(JSON.stringify(regStore)); 
        
        let xdata = xyData['xData'].filter(Number); // filter in case null values are present in array
        let ydata = xyData['yData'].filter(Number);
        let hdata = xyData['hoverData'];
        
        let slope = parseFloat(regData['slope']); 
        let intercept = parseFloat(regData['intercept']);
        
        let xline = [Math.min(...xdata), Math.max(...xdata)]; // xdata for a LOBF display
        let yline = xline.map(function(x) { return x*slope + intercept;}).map(Number);
        
        console.log(xline);
        console.log(yline);

        scatterCopy['data'][0]['x'] = xdata;
        scatterCopy['data'][0]['y'] = ydata;
        scatterCopy['data'][0]['hovertext'] = hdata;
        
        sizeArr = Array(xdata.length).fill(25)
        
        scatterCopy['data'][0]['marker'] = {'color': '#4F61E4', 'line':{'color':'black'}, 'size':sizeArr}; 
        scatterCopy['layout']['xaxis']['title'] = {'text':xVar, 'font':{'size':22}}; 
        scatterCopy['layout']['yaxis']['title'] = {'text':yVar, 'font':{'size':22}};
        
        scatterCopy['data'][1]['x'] = xline;
        scatterCopy['data'][1]['y'] = yline;

        return scatterCopy;

    
    }
    """, 
    Output(component_id = 'scatterPlot', component_property = 'figure'), 
    Input(component_id  = 'regStore', component_property = 'data'), 
    State(component_id  = 'xyStore', component_property = 'data'),
    State(component_id  = 'graphBool', component_property = 'value'), 
    State(component_id  = 'scatterPlot', component_property = 'figure'), 
    State(component_id  = 'xdat', component_property = 'value'), 
    State(component_id  = 'ydat', component_property = 'value'), 

    prevent_initial_call = True)
    

In [74]:
app.clientside_callback(
    """
    function(xyData) {
        /* JS Code
            Calculate the the r-squared value for least squares regression and display it
        */
        let xdata = xyData['xData']; 
        let ydata = xyData['yData'];
        
        function calculateR2(x, y) {
            if (x.length != y.length) {
                throw new Error('Input arrays not same length'); 
            }
            
            const n = x.length; 
            
            const meanX = x.reduce((sum, value) => sum + value, 0)/n; 
            const meanY = y.reduce((sum, value) => sum + value, 0)/n; 
            
            
            let numerator = 0; 
            let denominatorX = 0; 
            let denominatorY = 0; 
            
            for (let i = 0; i < n; i++) {
                numerator += (x[i] - meanX)*(y[i] - meanY);
                denominatorX += Math.pow(x[i] - meanX, 2); 
                denominatorY += Math.pow(y[i] - meanY, 2);
            }
            
            const rSquared = Math.pow(numerator, 2) / (denominatorX * denominatorY); 
            
            return rSquared;
        }
                
    let r2 = parseFloat(calculateR2(xdata, ydata).toPrecision(4));
    return 'R-squared:' + r2;
   
    }
    """, 
    
    Output(component_id = 'r2display', component_property = 'children'), 
    Input(component_id = 'xyStore', component_property  = 'data'), 
    prevent_initial_call = True)

In [75]:
app.clientside_callback(
    """
    function(xyData) { 
    
        /* JS Code
            Compute the slope and intercept of the LOBF and store it
        */ 
        
        let xdata = xyData['xData'];
        let ydata = xyData['yData'];
        
        function calculateSlope(x,y) { 
            if (x.length != y.length) { 
                throw new Error('Input arrays not same length'); 
            }
            
            const n = x.length; 
            const meanX = x.reduce((sum, value) => sum + value, 0)/n; 
            const meanY = x.reduce((sum, value) => sum + value, 0)/n;
            
            let numerator = 0; 
            let denominator = 0; 
            
            for (let i = 0; i <n; i++) {
                numerator += (x[i] - meanX)*(y[i] - meanY); 
                denominator += Math.pow((x[i] - meanX), 2); 
            }
            
            const slope = numerator/denominator; 
        
            return slope; 
        }
        
        let m = calculateSlope(xdata, ydata).toPrecision(3); 
        
        function calculateIntercept(x,y, m) {
            if (x.length != y.length) { 
                throw new Error('Input arrays not same length'); 
            }
            
            const n = x.length; 
            
            const sumX = x.reduce((sum, value) => sum + value, 0); 
            const sumY = y.reduce((sum, value) => sum + value, 0); 
            
            
            return (sumY - m*sumX)/n;
        }
        
        let b = calculateIntercept(xdata, ydata, m).toPrecision(3); 
        
        return {'slope': m, 'intercept': b}
    }
    """, 
    
    Output(component_id = 'regStore', component_property = 'data'), 
    Input(component_id  = 'xyStore', component_property = 'data'), 
    prevent_initial_call = True)

In [76]:
app.clientside_callback(
    """
    function(regData) { 
    
        /* JS Code
            Update the slope display
        */ 
        
        let slope = regData['slope']; 
        let intercept = regData['intercept']; 
        
        return 'y = ' + parseFloat(slope) + 'x + ' + parseFloat(intercept); 
    }
        
    """, 
    Output(component_id = 'lineDisplay', component_property = 'children'), 
    Input(component_id  = 'regStore', component_property = 'data'), 
    prevent_initial_call = True)

In [77]:
# outputs = {'oldVars': 'dropVars', 'oldMetric': 'dropMetric', 'oldMethod': 'dropMethod', 'oldComp': 'dropComp', 'oldRegion': 'previous_click-store', 'oldNorm': 'dtype'}

# for output, state in outputs.items(): 
#     if state == 'previous_click-store': 
#         prop = 'data'
#     else: 
#         prop = 'value'
        
#     app.clientside_callback(
#         """
#         // update the "old" parameters after the table data has finished updating
#         function(tableUpdate, input) {
#             return input;
        
#         }
#         """, 
#         Output(component_id = output, component_property='data'), 
#         Input(component_id = 'tbl', component_property = 'data'), 
#         State(component_id = state, component_property = prop), 
#         prevent_initial_call = True)

In [78]:
if __name__ == '__main__':
    app.run_server(debug = False, port = 8079, use_reloader = False)

Dash is running on http://127.0.0.1:8079/



2024-01-30 15:56:59,113 [INFO]: dash.py(run:1977) >> Dash is running on http://127.0.0.1:8079/

2024-01-30 15:56:59,113 [INFO]: dash.py(run:1977) >> Dash is running on http://127.0.0.1:8079/

2024-01-30 15:56:59,119 [INFO]: _internal.py(_log:224) >> WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8079
2024-01-30 15:56:59,119 [INFO]: _internal.py(_log:224) >> WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8079
2024-01-30 15:56:59,120 [INFO]: _internal.py(_log:224) >> Press CTRL+C to quit
2024-01-30 15:56:59,120 [INFO]: _internal.py(_log:224) >> Press CTRL+C to quit
2024-01-30 15:56:59,720 [INFO]: _internal.py(_log:224) >> 127.0.0.1 - - [30/Jan/2024 15:56:59] "GET /_alive_37823309-7928-4a84-9457-c0bbe6318a8d HTTP/1.1" 200 -
2024-01-30 15:56:59,720 [INFO]: _internal.py(_log:224) >> 127.0

Dash app running on http://127.0.0.1:8079/
